## DATA ANALYTICS – IBM PROJECT

Why do workers resign from the positions they hold? Can employee attrition be anticipated? Can companies take steps to prevent high employee turn-over?

Employee attrition is the reduction of staff by voluntary or involuntary reasons. These can be through natural means like retirement, or it can be through resignation, termination of contract, or when a company decides to make a position redundant. Excessive involuntary attrition can be detrimental to the overall business goals, and can lead to understaffing, increased job burden on remaining employees, and harmful consequences to company reputation. 

You’ve just been hired by IBM to work in their Data Analytics sector. On your second day, you are given a dataset from HR that includes several records of employees that were with the company in the prior calendar year. Since the development of the dataset, some of those employees have since resigned from their positions. The company was surprised with the recent level of turnover and have tasked you with digging into the data to determine any insights about what leads to employee turnover, and alternatively, what aspects of the jobs can be protected to reduce employee turnover. 

Using the ‘IBM Employee Attrition’ dataset – your role is to comprehensively explore the data and generate meaningful insights into what the data is presenting. 

You should conclude your assignment with 3 recommendations you will make to the HR department regarding employee attrition. 

At the least, you should complete the following tasks:
    
    •Data prep and cleaning – prior to exploring the data, you should ensure that the dataset is clean, all variables are in their proper format, and the data is free of missing, abnormal, or outlier values. 
    
    •Exploratory data analysis – you should explore every aspect of the data to determine the features of the data, the range of values for all variables of interest, and the relationships between variables. Exploration should be conducted outside of predictive modeling – in other words, you are not only interested in how the dependent and independent variables interact – you should also spend some time exploring variations in the data among variables (i.e. how does salary differ between genders? Does time spent in the company differ by job role?)
    
    •Modeling – after thoroughly exploring the data, create a logistic regression model that predicts employee attrition based on the other variables available to you. Clearly interpret your model findings. Re-run/tweak your model if you think you can find a better fit. 
    
    •Visualization – Create at least 3 visualizations that highlight some of the most interesting findings in your dataset. These can be findings from the regression model or any interesting relationships between variables. 

Good luck!


In [1]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as sm
import scipy.stats as stats

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns

###
df = pd.read_excel("IBM Employee Attrition.xlsx")
df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,1,2,Female,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,2,3,Male,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,4,Male,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,5,4,Female,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,7,1,Male,...,3,4,1,6,3,3,2,2,2,2


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   Department                1470 non-null   object
 4   DistanceFromHome          1470 non-null   int64 
 5   Education                 1470 non-null   int64 
 6   EducationField            1470 non-null   object
 7   EmployeeNumber            1470 non-null   int64 
 8   EnvironmentSatisfaction   1470 non-null   int64 
 9   Gender                    1470 non-null   object
 10  JobInvolvement            1470 non-null   int64 
 11  JobLevel                  1470 non-null   int64 
 12  JobRole                   1470 non-null   object
 13  JobSatisfaction           1470 non-null   int64 
 14  MaritalStatus           

In [ ]:
## 30 columns and 1470 employees' data

In [ ]:
df.isnull().sum()

In [ ]:
df["Over18"].value_counts()  ## looks like everyone is over 18, remove redundant column

In [8]:
df.drop("Over18", axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   Department                1470 non-null   object
 4   DistanceFromHome          1470 non-null   int64 
 5   Education                 1470 non-null   int64 
 6   EducationField            1470 non-null   object
 7   EmployeeNumber            1470 non-null   int64 
 8   EnvironmentSatisfaction   1470 non-null   int64 
 9   Gender                    1470 non-null   object
 10  JobInvolvement            1470 non-null   int64 
 11  JobLevel                  1470 non-null   int64 
 12  JobRole                   1470 non-null   object
 13  JobSatisfaction           1470 non-null   int64 
 14  MaritalStatus           

In [3]:
## function to calculate z_scores. Accepts 2 arguments, the dataframe and the column x for z-score
## returns the out_liers index

def Cal_zscore(df_temp, x):
    y = "zscore_" + x
    df_temp[y] = np.abs(stats.zscore(df_temp[x]))

In [14]:
dfz = df.copy()  ## copy dataframe to calculate outliers by zscore

In [15]:
dfz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   Department                1470 non-null   object
 4   DistanceFromHome          1470 non-null   int64 
 5   Education                 1470 non-null   int64 
 6   EducationField            1470 non-null   object
 7   EmployeeNumber            1470 non-null   int64 
 8   EnvironmentSatisfaction   1470 non-null   int64 
 9   Gender                    1470 non-null   object
 10  JobInvolvement            1470 non-null   int64 
 11  JobLevel                  1470 non-null   int64 
 12  JobRole                   1470 non-null   object
 13  JobSatisfaction           1470 non-null   int64 
 14  MaritalStatus           

In [5]:
dfq = df.copy()  ## copy dataframe to calculate outliers by IQR

In [ ]:
dfq.info()

In [16]:
## get each column, check if column has only numbers
## call Cal_zscore function to create columns zscore

for i in dfz:
    if np.issubdtype(dfz[i].dtype, np.number):
        Cal_zscore(dfz, i)

In [17]:
dfz.info()  ## removed 136 employees because they were outliers

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 50 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              1470 non-null   int64  
 1   Attrition                        1470 non-null   object 
 2   BusinessTravel                   1470 non-null   object 
 3   Department                       1470 non-null   object 
 4   DistanceFromHome                 1470 non-null   int64  
 5   Education                        1470 non-null   int64  
 6   EducationField                   1470 non-null   object 
 7   EmployeeNumber                   1470 non-null   int64  
 8   EnvironmentSatisfaction          1470 non-null   int64  
 9   Gender                           1470 non-null   object 
 10  JobInvolvement                   1470 non-null   int64  
 11  JobLevel                         1470 non-null   int64  
 12  JobRole             

In [ ]:
i = 29

while i <= 49:
    z_outliers = dfz.iloc[dfz[:,i] > 3].index
    i= i + 1
    print(z_outliers)

## Preview list of index values


In [23]:
dfz.iloc[:,29]

0       0.446350
1       1.322365
2       0.008343
3       0.429664
4       1.086676
          ...   
1465    0.101159
1466    0.227347
1467    1.086676
1468    1.322365
1469    0.320163
Name: zscore_Age, Length: 1470, dtype: float64

In [29]:
z_outliers = dfz.loc[dfz["zscore_Age"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [30]:
z_outliers = dfz.loc[dfz["zscore_DistanceFromHome"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [31]:
z_outliers = dfz.loc[dfz["zscore_Education"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [32]:
z_outliers = dfz.loc[dfz["zscore_EmployeeNumber"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [33]:
z_outliers = dfz.loc[dfz["zscore_EnvironmentSatisfaction"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [34]:
z_outliers = dfz.loc[dfz["zscore_JobInvolvement"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [35]:
z_outliers = dfz.loc[dfz["zscore_JobLevel"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [36]:
z_outliers = dfz.loc[dfz["zscore_JobSatisfaction"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [37]:
z_outliers = dfz.loc[dfz["zscore_MonthlyIncome"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [38]:
z_outliers = dfz.loc[dfz["zscore_NumCompaniesWorked"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [39]:
z_outliers = dfz.loc[dfz["zscore_PercentSalaryHike"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [40]:
z_outliers = dfz.loc[dfz["zscore_PerformanceRating"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [41]:
z_outliers = dfz.loc[dfz["zscore_RelationshipSatisfaction"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [42]:
z_outliers = dfz.loc[dfz["zscore_StockOptionLevel"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [43]:
z_outliers = dfz.loc[dfz["zscore_TotalWorkingYears"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([  85,   98,  126,  187,  270,  401,  445,  595,  624,  894,  956,
            1043, 1116, 1184, 1301, 1401],
           dtype='int64')


In [44]:
z_outliers = dfz.loc[dfz["zscore_TrainingTimesLastYear"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [45]:
z_outliers = dfz.loc[dfz["zscore_WorkLifeBalance"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([], dtype='int64')


In [46]:
z_outliers = dfz.loc[dfz["zscore_YearsAtCompany"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([  62,   98,  126,  190,  237,  270,  411,  425,  473,  477,  561,
             592,  595,  653,  677,  749,  861,  914,  918,  962, 1086, 1111,
            1116, 1135, 1138],
           dtype='int64')


In [47]:
z_outliers = dfz.loc[dfz["zscore_YearsInCurrentRole"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([63, 123, 190, 231, 281, 466, 716, 746, 976, 1024, 1327, 1351,
            1430],
           dtype='int64')


In [48]:
z_outliers = dfz.loc[dfz["zscore_YearsSinceLastPromotion"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([  45,   62,   64,  110,  123,  126,  178,  218,  237,  326,  386,
             425,  427,  535,  584,  595,  653,  701,  752,  799,  838,  861,
             875,  914,  922,  926,  937,  976, 1008, 1078, 1086, 1093, 1138,
            1156, 1221, 1223, 1242, 1295, 1301, 1303, 1331, 1414],
           dtype='int64')


In [49]:
z_outliers = dfz.loc[dfz["zscore_YearsWithCurrManager"] > 3].index

## Preview list of index values
print(z_outliers)

Int64Index([28, 123, 153, 187, 231, 386, 561, 616, 635, 686, 875, 926, 1078,
            1348],
           dtype='int64')


In [51]:
dfz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 50 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              1470 non-null   int64  
 1   Attrition                        1470 non-null   object 
 2   BusinessTravel                   1470 non-null   object 
 3   Department                       1470 non-null   object 
 4   DistanceFromHome                 1470 non-null   int64  
 5   Education                        1470 non-null   int64  
 6   EducationField                   1470 non-null   object 
 7   EmployeeNumber                   1470 non-null   int64  
 8   EnvironmentSatisfaction          1470 non-null   int64  
 9   Gender                           1470 non-null   object 
 10  JobInvolvement                   1470 non-null   int64  
 11  JobLevel                         1470 non-null   int64  
 12  JobRole             

In [54]:
dfz["YearsWithCurrManager"].value_counts()

2     344
0     263
7     216
3     142
8     107
4      98
1      76
9      64
5      31
6      29
10     27
11     22
12     18
13     14
17      7
15      5
14      5
16      2
Name: YearsWithCurrManager, dtype: int64

In [52]:
dfz.iloc[[28, 123, 153, 187, 231, 386, 561, 616, 635, 686, 875, 926, 1078,
            1348], 28]

28      17
123     15
153     15
187     15
231     15
386     17
561     16
616     17
635     15
686     17
875     17
926     17
1078    17
1348    16
Name: YearsWithCurrManager, dtype: int64

In [55]:
dfz["zscore_YearsSinceLastPromotion"].value_counts()

0.679146    581
0.368715    357
0.058285    159
1.493867     76
0.562576     61
0.252146     52
0.873006     45
1.183437     32
2.735589     24
1.804297     18
2.114728     17
3.977310     13
3.356449     10
3.046019     10
3.666880      9
2.425158      6
Name: zscore_YearsSinceLastPromotion, dtype: int64

In [57]:
dfz.iloc[[45,   62,   64,  110,  123,  126,  178,  218,  237,  326,  386,
             425,  427,  535,  584,  595,  653,  701,  752,  799,  838,  861,
             875,  914,  922,  926,  937,  976, 1008, 1078, 1086, 1093, 1138,
            1156, 1221, 1223, 1242, 1295, 1301, 1303, 1331, 1414], 27]

45      15
62      13
64      12
110     12
123     15
126     15
178     15
218     12
237     15
326     13
386     12
425     15
427     13
535     12
584     14
595     13
653     14
701     15
752     15
799     13
838     13
861     15
875     13
914     15
922     14
926     15
937     15
976     15
1008    12
1078    14
1086    13
1093    13
1138    12
1156    14
1221    14
1223    14
1242    12
1295    13
1301    14
1303    14
1331    12
1414    12
Name: YearsSinceLastPromotion, dtype: int64

In [ ]:
## Drop rows with above index values
dfz = dfz.drop(z_outliers)

## Re-check the shape of the dataframe, how many rows were dropped?
print(dfz.shape)